# Data Preparation



In [2]:
!cp '/content/drive/MyDrive/Meta_learning_research/Notebooks/data_util.py' .
from data_util import *

In [3]:
import os
import numpy as np
import tensorflow as tf

In [4]:
class MetaDataLoader:
    def __init__(self, data_dir, num_samples_per_location=100):
        self.data_dir = data_dir
        self.num_samples_per_location = num_samples_per_location

    def _load_and_process_data(self, locations):
        data_dict = {}

        for location in locations:
            location_dir = os.path.join(self.data_dir, location)

            train_data_path = os.path.join(location_dir, "train_data.npy")
            train_label_path = os.path.join(location_dir, "train_label.npy")
            vali_data_path = os.path.join(location_dir, "vali_data.npy")
            vali_label_path = os.path.join(location_dir, "vali_label.npy")

            # Load training data and labels
            train_data = np.load(train_data_path)
            train_label = np.load(train_label_path)

            # Load validation data and labels
            vali_data = np.load(vali_data_path)
            vali_label = np.load(vali_label_path)

            # Store data and labels in the data_dict with location as key
            data_dict[location] = {
                'train_data': train_data,
                'train_label': train_label,
                'vali_data': vali_data,
                'vali_label': vali_label
            }

        return data_dict

    def _create_episode(self, locations):

        data_dict = self._load_and_process_data(locations)

        #--------- Create the support set -------------
        selected_data = []
        selected_labels = []
        for location in locations:
            selected_data.extend(data_dict[location]['train_data'][:num_samples_per_location])
            selected_labels.extend(data_dict[location]['train_label'][:num_samples_per_location])


        data = np.array(selected_data)
        labels = np.array(selected_labels)

        # Shuffle the data and labels if needed
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        support_set_data = data[indices]
        support_set_labels = labels[indices]

        #--------- End create the support set -------------

        #--------- Create the query set -------------
        selected_data = []
        selected_labels = []

        for location in locations:
            selected_data.extend(data_dict[location]['vali_data'][:num_samples_per_location])
            selected_labels.extend(data_dict[location]['vali_label'][:num_samples_per_location])

        data = np.array(selected_data)
        labels = np.array(selected_labels)

        # Shuffle the data and labels if needed
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        query_set_data = data[indices]
        query_set_labels = labels[indices]

        #--------- End create the query set -------------

        return support_set_data, support_set_labels, query_set_data, query_set_labels

    def create_multi_episodes(self, num_episodes, locations):
        episodes = []
        for _ in range(num_episodes):
            support_set_data, support_set_labels, query_set_data, query_set_labels = self._create_episode(locations)
            episode = {
                "support_set_data": support_set_data,
                "support_set_labels": support_set_labels,
                "query_set_data": query_set_data,
                "query_set_labels": query_set_labels
            }
            episodes.append(episode)
        return episodes


In [4]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
data_dir = './samples/'  # Replace with the path to your directory containing numpy files
locations_meta_training = ['Alexander', 'Rowancreek']
locations_meta_testing = ['Covington']
num_samples_per_location = 100  # Configure the number of samples per location
num_episodes = 10  # Number of episodes

data_loader = MetaDataLoader(data_dir, num_samples_per_location)

# Create multi episodes for meta-training
episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_training)

/content/drive/MyDrive/Meta_learning_research/Notebooks


NameError: ignored

In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)


In [15]:
episodes[2]['query_set_labels'].shape

(130, 224, 224, 1)